<h1 style="font-size:2.8rem;color:#1351b4;">Modelo de Classificação de Propostas</h1>

O modelo de MVP observado junto ao cliente é um sistema classificador/categorizador das propostas publicadas na plataforma do [Brasil Participativo](https://brasilparticipativo.presidencia.gov.br/processes/programas/f/2/), sendo as categorias baseadas Ministérios do Brasil. Mais detalhes sobre os componentes do sistema abaixo.

Ademais, foi cogitado a possibilidade de adicionar componentes complementares ao pipeline do sistema, como um classificador de discurso de ódio e um classificador em subcategorias (baseadas nas próprias temáticas das propostas). Esses componentes ainda estão no campo das ideias.

<a id='1'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Sumário</p>

- [Extração dos dados](#data-extraction)
- [Pré-processamento textual](#pre-processing)
- [Treinamento do modelo](#training-model)

<a id='data-extraction'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Extração dos Dados</p>

<a id='pre-processing'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Pré-processamento Textual</p>

<a id='training-model'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Treinamento do Modelo</p>